In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上的端到端机器学习：MLOps阶段7：监控：XGBoost模型的Vertex AI模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_xgboost.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td> 
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_xgboost.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何为 XGBoost 模型使用`Vertex AI Model Monitoring`。

### 目标

在这本笔记本中，您将学习如何使用`Vertex AI Model Monitoring`服务来检测XGBoost模型输入预测请求中的特征偏差和漂移。

本教程使用以下谷歌云 ML 服务：

- `Vertex AI Model Monitoring`
- `Vertex AI Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

所执行的步骤包括：

- 下载一个预训练的XGBoost模型。
- 将预训练模型上传为`Model`资源。
- 将`Model`资源部署到`Endpoint`资源。
- 配置`Endpoint`资源用于模型监控：
  - 仅进行漂移检测 -- 无法访问训练数据。
  - 预定义输入模式，将特征别名映射到模型的未命名数组输入。
- 生成用于漂移检测的合成预测请求。

了解更多关于 [Introduction to Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用的模型是一个预训练的XGBoost模型，该模型是在[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)提供的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)上进行训练的。训练后的模型可以预测三种不同种类的鸢尾花品种：山鸢尾（setosa）、弗吉尼亚鸢尾（virginica）和变色鸢尾（versicolor）。

成本

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本电脑**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

**否则**，请确保您的环境满足此笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在带有Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了简化的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端命令行中运行 `pip install jupyter`。

5. 要启动Jupyter，在终端命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装所需的包以执行此笔记本。

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform \
                                             google-cloud-bigquery

### 重新启动内核

安装SDK后，您需要重新启动笔记本内核，以便它可以找到包。您可以从*Kernel -> Restart Kernel*重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得300美元的免费信用额用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保
Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 前缀开头的行作为 shell 命令运行，并将以 `$` 前缀开头的 Python 变量插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本的其他操作。以下是Vertex AI支持的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

**对于本笔记本，我们建议您将区域设置为默认值us-central1**。

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行直播教程会话，您可能会使用一个共享的测试账户或项目。为了避免用户在创建资源时发生名称冲突，您可以为每个实例会话创建一个 UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

####用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
USER_EMAIL = "[your-email-addr]"  # @param {type:"string"}

###验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经被验证。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务账户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账户**。

3. 在**服务账户名称**栏中输入一个名称，并点击**创建**。

4. 在**将此服务账户授予对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

5. 点击**创建**。包含您的密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中，将您的服务账户密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 关于服务账号和权限的注释

**默认情况下不需要任何配置**，如果遇到任何与权限相关的问题，请确保上述服务账号具有所需的角色：

|服务账号邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|计算引擎默认服务账号|Dataflow Admin, Dataflow Worker, Storage Admin, BigQuery Admin, Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI平台服务代理|Vertex AI服务代理|

1. 进入 https://console.cloud.google.com/iam-admin/iam。
2. 勾选“包括Google提供的角色授予”复选框。
3. 找到以上的邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请授予这两个服务账号“BigQuery Data Viewer”角色。
- 对于CSV数据源，请授予这两个服务账号“Storage Object Viewer”角色。

### 创建一个云存储桶

**无论您使用什么笔记本环境，下面的步骤都是必需的。**

在下面设置您的云存储桶的名称，您将在本教程中使用该名称上传监控服务的 `输入架构`。

存储桶的名称在所有谷歌云项目中必须是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对云存储存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 导入库

In [ ]:
from google.cloud import aiplatform, bigquery
from google.cloud.aiplatform import model_monitoring

### 为Python初始化Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### 创建 BigQuery 客户端

在本教程中，您将使用与训练预训练模型所使用的相同的公共 BigQuery 表中的数据。您将创建一个客户端界面，随后用于访问这些数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
DEPLOY_VERSION = "xgboost-cpu.1-1"

LOCATION = REGION.split("-")[0]

DEPLOY_IMAGE = f"{LOCATION}-docker.pkg.dev/vertex-ai/prediction/{DEPLOY_VERSION}:latest"

print("Deployment:", DEPLOY_IMAGE)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Vertex AI模型监控简介

Vertex AI模型监控支持AutoML表格模型和自定义表格模型。您可以监测入站预测请求中特征的偏斜和漂移检测，或者监测出站预测响应中特征归因（可解释人工智能）的偏斜和漂移检测 -- 即，归因分布对输出（预测）的贡献方式。

以下是启用模型监控的基本步骤：

1. 将`Vertex AI` AutoML或自定义表格模型部署到`Vertex AI端点`。
2. 配置模型监控规范。
3. 上传模型监控规范到`Vertex AI端点`。
4. 上传或自动生成`输入模式`用于解析。
5. 对于特征偏斜检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的`Vertex AI可解释性`规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控后，自动采样的入站预测请求将被记录在一个BigQuery表中。然后分析记录请求中包含的输入特征值，以检测特定间隔基础上的偏斜或漂移。您可以设置采样率来监测模型的生产输入的子集，以及监控间隔。

模型监控服务需要了解如何解析特征值，这被称为输入模式。对于AutoML表格模型，输入模式会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动推导输入模式。或者，您可以上传输入模式。

对于偏斜检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格模型，这会自动生成。对于自定义表格模型，您需要将训练数据上传到服务，并使服务自动推导分布。

对于特征归因偏斜和漂移检测，需要为自定义表格模型启用`Vertex AI可解释性`。对于AutoML模型，`Vertex AI可解释性`会自动启用。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 将模型工件上传为“Vertex AI Model”资源

首先，使用“upload（）”方法将预先训练的XGBoost表格模型工件作为“Vertex AI Model”资源上传，参数如下：

- `display_name`：模型资源的人类可读名称。
- `artifact_uri`：模型工件的Cloud Storage位置。
- `serving_container_image`：在模型部署到“Vertex AI Endpoint”资源时要使用的服务容器映像。
- `sync`：是否等待过程完成，还是立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = (
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/xgboost_iris"
)

model = aiplatform.Model.upload(
    display_name="xgboost_iris_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源

接下来，您可以使用`deploy()`方法将您的`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源，参数如下：

- `deploy_model_display`：部署模型的人类可读名称。
- `machine_type`：每个VM节点实例的机器类型。
- `min_replica_count`：自动扩展所需的最小节点数。
- `max_replica_count`：自动扩展所需的最大节点数。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name="xgboost_iris_" + UUID,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

## 配置监控任务

配置监控任务包括以下规范：

- `alert_config`：发送监控警报的电子邮件地址。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：采样预测请求的速率。
- `drift_config`：要监视的特征和漂移阈值。
- `skew_config`：要监视的特征和偏斜阈值。

### 配置警报规范

首先，您使用以下设置配置 `alerting_config` 规范：

- `user_emails`：一个或多个要发送警报的电子邮件列表。
- `enable_logging`：将检测到的异常流向 Cloud Logging。默认值为 False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您将使用以下设置配置`schedule_config`规范：

- `monitor_interval`: 设置模型监控作业调度间隔，单位为小时。最小时间间隔为1小时。例如，以一小时的间隔，监控作业将每小时运行一次。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置抽样规范

接下来，您可以使用以下设置来配置 `logging_sampling_strategy` 规范：

- `sample_rate`: 随机抽样预测请求监控的比例（在0到1之间的百分比）。选择的样本将被记录到一个 BigQuery 表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您将使用以下设置配置 `drift_config` 规范：

- `drift_thresholds`：一个包含键/值对的字典，其中键是要监视漂移的输入特征，值是检测阈值。如果未指定，则特征的默认漂移阈值为0.3（30%）。

*注意：*启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "sepal_length": DRIFT_THRESHOLD_VALUE,
    "petal_length": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 组装目标规范

最后，您将用以下设置组装目标规范`objective_config`：

- `skew_detection_config`:（可选）倾斜检测配置的规范。
- `drift_detection_config`:（可选）漂移检测配置的规范。
- `explanation_config`:（可选）在启用特征归因监控时的解释规范。

*注意：*您不需要配置倾斜检测，因为假设您无法访问训练数据。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=None,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型作为模型的输入，这被称为`输入模式`。`输入模式`可以是
 - 预加载到监控服务中。
 - 在接收到前1000个预测实例后，由监控服务自动生成。

在本教程中，您将预加载`输入模式`。

#### 创建预定义的输入模式

预定义的`输入模式`以YAML文件的形式指定。在本例中，您将根据模型的输入层生成YAML规范。在本例中，输入层是一个包含四个浮点数值的数组。在模式中，这表示为：

- `type: array`：表示输入是一个数组（列表）
- `properties`：数组中输入的有序列表
- `properties -> name`：数组中对应值的别名（例如，sepal_length）。
- `properties -> type: number`：数组元素的值为浮点数。
- `required`：由别名指定的数组中值的顺序。

然后，输入模式告诉模型监控服务如何将未命名的输入值映射到相应的特征别名，然后可以在您的模型监控配置中指定这些别名。

预定义的`输入模式`必须加载到云存储位置。

了解更多关于[解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
yaml = """type: array
properties:
  sepal_length:
    type: number
  sepal_width:
    type: number
  petal_length:
    type: number
  petal_width:
    type: number
required:
  - sepal_length
  - sepal_width
  - petal_length
  - petal_width
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控任务

您可以使用`aiplatform.ModelDeploymentMonitoringJob.create()`方法创建一个监控任务，设定监控规范，具体参数如下：

- `display_name`：监控任务的可读名称。
- `project`：项目ID。
- `region`：区域。
- `endpoint`：启用监控的`Vertex AI Endpoint`的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含`输入模式`的YAML文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="xgboost_iris_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

监控工作的电子邮件通知。

将电子邮件通知发送到警报配置中的电子邮件地址，通知模型监控工作现已启用。

内容将如下显示：

您好，Vertex AI 客户，

您收到此邮件是因为您正在使用 Vertex AI 模型监控服务。
此邮件是为了通知您，我们已收到您设置漂移或偏斜检测的请求，用于下面列出的预测端点。 从现在开始，传入的预测请求将被采样并记录以供分析。
原始请求和响应将从预测服务中收集，并保存在 bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict。

#### 监控作业状态

在启动 `Vertex AI Model Monitoring` 作业后，该作业可能处于三种转换状态之一：

- `PENDING`: 作业已配置为检测偏差，并正在计算`偏差分布基线`。监控服务将启动一个批处理作业来从训练数据生成分布基线。

- `OFFLINE`: 监控作业处于监控间隔之间的状态。

- `RUNNING`: 每个间隔基础上的监控作业正在分析抽样数据。

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=xgboost_iris_{UUID}")
job = jobs[0]
print(job.state)

暂停监控工作以启用

In [ ]:
import time

time.sleep(180)

### 为第一个基准线生成合成预测请求

接下来，您需要创建1000个合成数据项用于预测请求。

In [ ]:
instances = []
for _ in range(1000):
    sepal_length = random.uniform(0.5, 3.5)
    sepal_width = random.uniform(0.2, 2.0)
    petal_length = random.uniform(0.5, 2.0)
    petal_width = random.uniform(0.2, 1.5)
    instances.append([sepal_length, sepal_width, petal_length, petal_width])

### 发出预测请求

接下来，您可以使用`predict()`方法将1000个预测请求发送到您的`Vertex AI端点`资源。

请注意，模型输出的类别是浮点值。例如，`0.0`代表标签`0`。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在下一个监控周期内，抽样的预测结果将被复制到BigQuery日志表中。一旦条目在BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待第一个记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，因此您应该会看到大约500个条目。

*注意*: 这可能需要等待监控周期的长度（例如，一个小时）。

In [ ]:
import time

while True:

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break
    time.sleep(180)

### 生成用于漂移检测的合成预测请求

您可以修改数据（合成数据）以触发漂移检测，从而比较上一个基线分布与当前分布的预测请求，具体操作如下：

- `sepal_length`：将数值增加4倍。

In [ ]:
instances = []
for _ in range(1000):
    sepal_length = random.uniform(0.5, 3.5) * 4.0
    sepal_width = random.uniform(0.2, 2.0)
    petal_length = random.uniform(0.5, 2.0)
    petal_width = random.uniform(0.2, 1.5)
    instances.append([sepal_length, sepal_width, petal_length, petal_width])

### 发送预测请求

接下来，您可以使用 `predict()` 方法向您的 `Vertex AI Endpoint` 资源发送 1000 个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 监控过程中的漂移检测

特征输入漂移检测将在下一个监控间隔期间发生。在本教程中，您将监控间隔设置为一个小时。所以大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行时，它将分析此间隔内预测的已记录样本表，并将它们与先前监控间隔的分布进行比较。

一旦分析完成，监控作业将发送电子邮件通知检测到的漂移，本例中为“cnt_user_engagement”，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔发生分析到您收到电子邮件警报可能需要多达40分钟。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 记录抽样请求

在下一个监控间隔中，抽样的预测将被复制到 BigQuery 的日志表中。一旦条目在 BigQuery 表中，监控服务将分析抽样数据。

接下来，您等待第二个已记录的条目出现在用于记录预测样本的 BigQuery 表中。由于您发送了1000次预测请求，抽样率为50％，您应该看到大约1000个条目。

*注意*：这可能需要等待监控间隔的长度（例如，一个小时）。

In [ ]:
import time

while True:

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 950:
        break
    time.sleep(180)

### 删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署和删除 `Vertex AI Endpoint` 资源

您可以使用 `delete()` 方法删除您的 `Vertex AI Endpoint` 资源。在删除之前，部署到您的 `Vertex AI Endpoint` 资源的任何模型必须先取消部署。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

## 清理工作

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}